<a target="_blank" href="https://colab.research.google.com/github/huseyinegg/IE423-2024SUMMER/blob/main/M.HuseyinBeyribey_IE423_TASK7.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

dfBFriday = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/black_friday.csv')
dfBFriday.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(550068, 12)

In [2]:
dfBFriday_idbased = pd.DataFrame(columns=["User_ID", "Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status", "Total_Purchase"])
dfBFriday_idbased = dfBFriday.groupby("User_ID").agg({"Gender":"first", "Age":"first", "Occupation":"first", "City_Category":"first", "Stay_In_Current_City_Years":"first", "Marital_Status":"first", "Purchase":"sum"})
dfBFriday_idbased.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Purchase
User_ID,,,,,,,
1000001,F,0-17,10,A,2,0,334093
1000002,M,55+,16,C,4+,0,810472
1000003,M,26-35,15,A,3,0,341635
1000004,M,46-50,7,B,2,1,206468
1000005,M,26-35,20,A,1,1,821001


# Association Rule Mining

Support = how likely is it for both items to show up together

Confidence = how likely is it for D to show up given A showed up

Lift = Confidence compared to D showing up anyway

In [3]:
dfBFriday['clean_description'] = dfBFriday['Product_ID']
dfBFridayList =dfBFriday.groupby('User_ID').clean_description.apply(list)
dfBFridayList

User_ID
1000001    [P00069042, P00248942, P00087842, P00085442, P...
1000002    [P00285442, P00112842, P00293242, P00289342, P...
1000003    [P00193542, P00132842, P0098342, P00010242, P0...
1000004    [P00184942, P00346142, P0097242, P00046742, P0...
1000005    [P00274942, P00251242, P00014542, P00031342, P...
                                 ...                        
1006036    [P00237642, P00250342, P00173242, P00255342, P...
1006037    [P00067342, P00111842, P00323642, P00177442, P...
1006038    [P00034742, P00251242, P00086042, P00109542, P...
1006039    [P00114042, P00199442, P00088542, P00254242, P...
1006040    [P00184042, P00158742, P00001742, P00148642, P...
Name: clean_description, Length: 5891, dtype: object

### apriori

In [4]:
!pip3 install mlxtend --upgrade
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [6]:
%time
te = TransactionEncoder()
te_ary = te.fit(dfBFridayList).transform(dfBFridayList)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

df_encoded

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,P0098942,P0099042,P0099142,P0099242,P0099342,P0099442,P0099642,P0099742,P0099842,P0099942
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,False,False,False,True,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
5887,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5888,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5889,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
%time
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


,support,itemsets
0,0.195553,(P00000142)
1,0.063826,(P00000242)
2,0.086912,(P00000642)
3,0.085384,(P00001042)
4,0.098116,(P00001142)
...,...,...
2143,0.050755,"(P00334242, P00355142)"
2144,0.050416,"(P00110742, P00057642, P00025442)"
2145,0.054660,"(P00110742, P00112142, P00025442)"
2146,0.050246,"(P00237542, P00110742, P00025442)"


In [16]:
apriori_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.55)
apriori_rules.sort_values(by='confidence', ascending=False, inplace=True)
apriori_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(P00270942, P00145042)",(P00057642),0.090816,0.249533,0.051095,0.562617,2.254677,0.028433,1.715811,0.612063


Confidence greater than 0.55 only for 1 instance.

In [18]:
apriori_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
apriori_rules.sort_values(by='confidence', ascending=False, inplace=True)
apriori_rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
26,"(P00270942, P00145042)",(P00057642),0.090816,0.249533,0.051095,0.562617,2.254677,0.028433,1.715811,0.612063
14,(P00329542),(P00114942),0.120183,0.199966,0.063317,0.526836,2.634628,0.039284,1.690818,0.705192
16,(P00129342),(P00145042),0.096079,0.238669,0.050246,0.522968,2.191185,0.027315,1.595975,0.601408
2,(P00346142),(P00025442),0.100662,0.274147,0.052623,0.522766,1.906881,0.025026,1.520957,0.528815
24,"(P00237542, P00110742)",(P00025442),0.096249,0.274147,0.050246,0.522046,1.904255,0.023860,1.518667,0.525432


### fpgrowth algorithm

In [19]:
from mlxtend.frequent_patterns import fpgrowth

%time
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.53 µs


,support,itemsets
0,0.274147,(P00025442)
1,0.244441,(P00184942)
2,0.238669,(P00059442)
3,0.234765,(P00255842)
4,0.231540,(P00117942)
...,...,...
1351677,0.010525,"(P00190942, P00112142)"
1351678,0.010015,"(P00190942, P00190142)"
1351679,0.010355,"(P00035542, P00190942)"
1351680,0.010355,"(P00046742, P00190942)"


In [20]:
fp_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
fp_rules.sort_values(by=['confidence'], ascending=False, inplace=True)

fp_rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
57,"(P00329542, P00128942, P00144642)",(P00057642),0.013071,0.249533,0.010185,0.779221,3.122714,0.006923,3.399173,0.688768
182,"(P00021742, P00147942, P00112142)",(P00110742),0.013410,0.273638,0.010355,0.772152,2.821803,0.006685,3.187923,0.654392
34,"(P00070042, P00277642, P00117942)",(P00145042),0.015617,0.238669,0.012052,0.771739,3.233510,0.008325,3.335354,0.701697
134,"(P00127742, P00058042, P00102642)",(P00057642),0.013750,0.249533,0.010525,0.765432,3.067456,0.007094,3.199359,0.683393
186,"(P00021742, P00112142, P00057942)",(P00110742),0.013580,0.273638,0.010355,0.762500,2.786531,0.006639,3.058367,0.649957


Examining the metrics for the first row, we notice that while the support is low, the confidence is high. This indicates that although the two items rarely appeared together, the presence of the antecedent strongly correlated with the presence of the consequent. Additionally, the low lift value suggests that the consequent is unlikely to occur without the antecedent.